In [2]:
import sys
import os
sys.path.append("../../src/utils/")
from read_shapefiles import read_files

import pandas as pd
import plotly as py
import plotly.graph_objs as go
from shapely.geometry import Point
import geopandas as gpd

In [16]:
def read_tusc(path):
    """
    Helper function to read in the Tuscany shapefile
    """
    path_shapefiles, regions, provinces, territories, municipalities, crs = read_files.read_shapefile_data(path, 'shape_files_path.json')
    df_reg_tus = read_files.read_shapefiles_in(True, path_shapefiles, regions, crs)
    return df_reg_tus


def filter_tusc(result):
    """
    Filter geopoints to get only Tuscany
    """
    geometry = [Point(xy) for xy in zip(result['avg_lon'], result['avg_lat'])]
    df_reg_tus=read_tusc("../../src/utils/read_shapefiles/")
    geo_cluster = gpd.GeoDataFrame(result, crs=df_reg_tus.crs, geometry=geometry)
    tusc_mask = geo_cluster.within(df_reg_tus.loc[8, 'geometry'])
    result_tusc = geo_cluster.loc[tusc_mask]
    return result_tusc

def create_file_name_and_path(country, season):
    path='../../results/kmeans/'
    file_name=country+"_"+season
    newpath=path+file_name+'/'
    if not os.path.exists(newpath):
        os.makedirs(newpath)
    return newpath,file_name

def create_data(result, names, colors):
    """
    Create data for visulaization
    """
    
    data=[]
    df=filter_tusc(result)
    clusters=df['label'].value_counts().index
    for i in clusters:
        print(i)
        x=df[df['label']==i][['avg_lat', 'avg_lon', 'label']]
        site_lat = x.avg_lat
        site_lon = x.avg_lon
        locations_name = x.index
        data.append(go.Scattermapbox(
            lat=site_lat,
            lon=site_lon,
            mode='markers',
            marker=dict(
                size=8,
                color=colors[int(i)],
                opacity=0.5
            ),
            text=locations_name,
            hoverinfo='none',
            name=names[int(i)])
        )
    return data

def plot_kmeans(result, names, colors, country, season, mapbox_access_token):
    d=create_data(result, names, colors)
    layout = go.Layout(
    title=country.title()+" in the "+season.title(),
    autosize=True,
    hovermode='closest',
    showlegend=True,
    mapbox=dict(
        accesstoken=mapbox_access_token,
        bearing=0,
        center=dict(
            lat=43.45089368,
            lon=11.12636257
        ),
        pitch=0,
        zoom=7,
        style='light'
    ),
)

    fig = dict(data=d, layout=layout)
    path,filename=create_file_name_and_path(country, season)
    print(path,filename)
    py.offline.iplot()
    py.offline.plot(fig, filename=path+filename+'.html')
    return fig

In [19]:
df_reg_tus=read_tusc("../../src/utils/read_shapefiles/")
names=['Explorers','East Tuscany','City hoppers']
colors=['rgb(255,255,0)', 'rgb(34,139,34)', 'rgb(220,20,60)', 'rgb(70,130,180)', 'rgb(128,0,128)']
mapbox_access_token='pk.eyJ1IjoidmFzYXJoZWx5aW8iLCJhIjoiY2prYjV2djh0M2R3NDNxbWw3dTFqdGZvbyJ9.stZ2MjMsogAYJ9fMb-lrsg'
result=pd.read_csv('../../results/kmeans/hungary_winter/hungary_winter.csv')
country='hungary'
season='winter'

f=plot_kmeans(result, names, colors, country, season, mapbox_access_token)

0.0
2.0
1.0
../../results/kmeans/hungary_winter/ hungary_winter


In [26]:
init_notebook_mode(connected=True)

iplot(f)